Setting up probes:

In [3]:
# desired DNA strand
MBP_seq = "AAAGACAGGCCCTCAGAGTCCGACGAGCTTCTCAGAGTCCGACGAGCTTCAGACCATCCAAGAAGATCCCACAGCAGCTTCCGAAGGCCTGGATGTGATGGCATCACAGAAGAGACCCTCACAGCGACACGGGTCCAAGTACTTGGCCACAGCAAGTACCATGGACCATGCCCGGCATGGCTTCCTCCCAAGGCACAGAGACACGGGCATCCTTGACTCCATCGGGCGCTTCTTTAGCGGTGACAGGGGTGCGCCCAAGCGGGGCTCTGGCAAGGTACCCTGGCTAAAGCAGAGCCGGAGCCCTCTGCCTTCTCATGCCCGCAGCCGTCCCGGGCTGTGCCACATGTACAAGGACTCACACACAAGAACTACCCACTACGGCTCCCTGCCCCAGAAGTCGCAGAGGACCCAAGATGAAAACCCAGTAGTCCACTTCTTCAAGAACATTGTGACACCTCGTACACCCCCTCCATCCCAAGGAAAGGGGAGAGGCCTGTCCCTCAGCAGATTTAGCTGGGGGGCCGAGGGGCAGAAGCCAGGATTTGGCTACGGAGGCAGAGCTTCCGACTATAAATCGGCTCACAAGGGATTCAAGGGGGCCTACGACGCCCAGGGCACGCTTTCCAAAATCTTTAAGCTGGGAGGAAGAGACAGCCGCTCTGGATCTCCCATGGCAAGACGCTGAGAGCCTCCCTGCTCAGCCTTCCCGAATCCTGCCCTCGGCTTCTTAATATAACTGCCTTAAACGTTTAATTCTACTTGCACCAAATAGCTAGTTAGAGCAGACCCTCTCTTAATCCCGTGGGGCTGTGAACGCGGCGGGGCCAGGCCCACGGCACCCTGACTGGCTAAAACTGTTTGTCCCTTTTTATTTGAAGATTGAGTTTCCTCGGGGTCTTCTCTGCCCCGACTTGCTCCCCGTGTACCTTGGTCGACTCCGGAGGTTCAGGTGCACGGACACCCTTTCAAGTTCACCCCTACTCCATCCTCAGACTTTCTTTTCACGGCGAGGCGCACCCCTCCAGCTTCCGTGGGCACTGCGGATAGACAGGCACACCGCCAAGGAGCCAGAGAGCATGGCGCAGGGGACTGTGTGGTCCAGGCTTCCTTTGTTTTCTTTCCCCTAAAGAGCTTTGTTTTTCCTAACAGGATCAGACAGTCTTGGAGTGGCTTACACAACGGGGGCTTGTGGTATGTGAGCACAGGCTGGGCAGCTGTGAGAGTCCAGAGTGGGGTGGCCCTGGGGACGCTTCCAGGCCAGCGGTTCCCTGCACCCCACCAGCTGATTTCGAGCGTGGCAGAGGGAAGGAAAGGGGCGAGCGGGCTGGGCAATGGACCCGACAGGAAACGGGGACTTAGGGGAACACGCTGGAGATGCCATGTGTGGCTGCCGAAGGTCACCATCTCTCCTCAGTGGCTCCCCAGAGCAGGTGCTTTTAAGAACCCTGTTTCCTCTCAGAGCCCAGGGAGAGTCCAAGGACATGGCGCATCAGGAAGTGGGACTGCAGGAGTTCTCTGGTGGCCTCGTGCTGTCCCTCTGGCCACTTCTCACTTTAGGGTGGTCAGCGGCAGCTCGCCATGGCAGTGCCCATTGGTGCACACTAACCTCAGTGGAAAAGTAACCATTCCCTGCCTCTTAGAAAGAACTCATTCTTAGTTTTAGGAGGGTTCCTGTCGCTGAATCAAGTCGCTGCCCTGGATGCAGGGCTGGCCTGGGCGACCCTCCAGGGATGAGGAGCTCAGAATTCCAGTCTTCTAATGTCCACGGACACCTCCCCATCCCTCTAACGTACTGACTATGTCTTTTGATTTAGCATGTCTTCTATAGACCTTCCAAAGAGACCCACACTGGCACTGTCACCCCCTAGGAGGGAAGGTGATGGTTGATGTAGCCCGACGCGCATCTTGTTAATCCGTTCTAATTCCGAGGAGAGTGTGGGTTTAAGATAACACCTATTAATGCATTGCCACAATAATGTGGGGGTAAGAGAAACGCAGGGACGAAACTTCCAGAAACAAACCCTCCAGATCGTTCCACAGGAGTGTTCGCCCTCCGGTGTGACTGAACGACCGACCTTGCCCATGGCTTCATCCAGACAGCACAGCTGCAGTATGGCTGGACAGAAGCACCTACTGTTCTTGGATATTGAAATAAAATAATAAACTTGCAATGATCTTTG"

In [5]:
# complement and reverse DNA strand
comp = MBP_seq.lower()[::-1].replace("g", "C").replace("c", "G").replace("t", "A").replace("a", "T")

In [6]:
# splitting probes
split_probes = []
for n in range(20, 31): 
    for i in range(0, len(comp), n):
        probe = comp[i:i + n]
        
        
        if len(probe) > 20:
            split_probes.append(probe)

# Not sure if we should include file below

# f = open("split_probes.txt", "w")
# f.write(str(split_probes))
# f.close()

PNAS rules:

In [7]:
# PNAS 1
def pnas_filter_1(probe_sequence):
    percent_a = probe_sequence.count('A')/len(probe_sequence)
    return 0.28 < percent_a

In [8]:
# PNAS 2
def pnas_filter_2(probe_sequence):
    A_stack = 'AAAA'
    if A_stack in probe_sequence:
        return False 
    else:
        return True

In [9]:
# PNAS 3
def pnas_filter_3(probe_sequence):
    percent_c = probe_sequence.count('C')/len(probe_sequence)
    return 0.22 < percent_c and 0.28 > percent_c

In [10]:
# PNAS 4
def pnas_filter_4(probe_sequence):
    C_stack = 'CCCC'
    sequence_subset = probe_sequence[0:12]
    if C_stack in sequence_subset:
        return False
    else:
        return True  

In [11]:
# PNAS 5
def pnas_filter_5(probe_sequence):
    sequence_subset = probe_sequence[0:12]
    for p in range(6):
        if sequence_subset[p:p+6].count('C') >= 4:
            return False
    return True

deltaG:

In [6]:
def deltaG1(probe_sequence):
    import pandas as pd
    
    table = {"AA":-0.2, "AC":-1.5, "AG":-0.9, "AT":-1.0,
        "CA":-1.0, "CC":-2.2, "CG":-1.2, "CT":-1.4,
        "GA":-0.8, "GC":-2.4, "GG":-1.5, "GT":-1.0, 
        "TA":-0.3, "TC":-1.4, "TG":-1.0, "TT":-0.4}
    
    for p in range(len(probe_sequence)):
        list_of_pairs = [probe_sequence[p:2 + p] for p in range(0, len(probe_sequence))]
        list_of_pairs.pop()
        if len(list_of_pairs) > 0:
            C = (pd.Series(list_of_pairs)).map(table)
            D = list(C)
            return sum(D)

In [7]:
deltaG1('AA')

-0.2

Percent of GC:

In [14]:
def GC_percent(probe_sequence):
    percent_GC = probe_sequence.count('G' and 'C')/len(probe_sequence)
    return percent_GC

Start and end indexes:

In [15]:
def start_index(probe_sequence):
    if probe in comp:
        start = comp.find(probe_sequence)
        return start

In [16]:
def end_index(probe_sequence):
    if probe in comp:
        end = comp.find(probe_sequence)
        end += len(probe_sequence) - 1
        return end